In [1]:
from model import Model

device = 'cuda:3'
model = Model(None, device=device)

In [19]:
# Load audio
import librosa
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import torch
import torchaudio
import librosa.display
import os

def load_audio(path):
    audio, sample_rate = torchaudio.load(path)
    return audio


DATA_PATH = './VCTK-synthesis-test-2_best'
audios = [load_audio(os.path.join(DATA_PATH, file)) for file in os.listdir(DATA_PATH)]

In [4]:
model.eval()

Model(
  (ssl_model): SSLModel(
    (model): Wav2Vec2Model(
      (feature_extractor): ConvFeatureExtractionModel(
        (conv_layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU()
          )
          (1): Sequential(
            (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU()
          )
          (2): Sequential(
            (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (1): Dropout(p=0.

In [20]:
audio = load_audio('common_voice_en_1164.wav')

In [22]:
import torch.nn.functional as F

output = model(audio)
probabilities = F.softmax(output, dim=1)
print(probabilities)
predicted_class = torch.argmax(probabilities, dim=1)

print(predicted_class)

tensor([[0.3755, 0.6245]], grad_fn=<SoftmaxBackward>)
tensor([1])


In [1]:
from torch.utils.data import Dataset
import torchaudio  # Assuming torchaudio is used for audio file loading
import os
from torch.utils.data import DataLoader

from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Sort the batch in the descending order of audio lengths
    batch.sort(key=lambda x: x[1], reverse=True)
    sequences, lengths, filenames = zip(*batch)

    # Pad the sequences to have equal length
    sequences_padded = pad_sequence(sequences, batch_first=True)
    
    # Debugging: print shapes
    print("Shapes before padding:", [s.shape for s in sequences])
    print("Shape after padding:", sequences_padded.shape)

    return sequences_padded, filenames

class CustomDataset(Dataset):
    def __init__(self, directory):
        self.directory = directory
        self.filenames = [f for f in os.listdir(directory) if f.endswith('.wav')]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filepath = os.path.join(self.directory, self.filenames[idx])
        waveform, _ = torchaudio.load(filepath)  # Load waveform
        return waveform, waveform.size(1), self.filenames[idx]
        
def produce_evaluation_file(dataset, model, device, save_path):
    data_loader = DataLoader(dataset, batch_size=14, shuffle=False, drop_last=False, collate_fn=collate_fn)
    model.eval()

    for batch_x, utt_id in data_loader:
        batch_x = batch_x.to(device)
        batch_out = model(batch_x)  # Modify as per your model's requirements
        
        # Assuming batch_out is the score or output you want to evaluate
        batch_score = batch_out.data.cpu().numpy().ravel()
        
        with open(save_path, 'a+') as fh:
            for f, score in zip(utt_id, batch_score):
                fh.write('{} {}\n'.format(f, score))


In [ ]:
from torch import nn
import torch
from model import Model

device = 'cuda' if torch.cuda.is_available() else 'cpu'                  
eval_output='eval_pre_trained_model_CM_scores_file_SSL_DF.txt'
model_path='./Best_LA_model_for_DF.pth'

model = Model(None, device=device)
model =nn.DataParallel(model).to(device)
database_path = './VCTK-synthesis-test-2_best'
model.load_state_dict(torch.load(model_path,map_location=device))


print('Device: {}'.format(device))
print('Model loaded : {}'.format(model_path))

custom_dataset = CustomDataset(directory=database_path)
produce_evaluation_file(custom_dataset, model, device, eval_output)

In [1]:
!CUDA_VISIBLE_DEVICES=0 python main_SSL_DF.py --track=DF --is_eval --eval --model_path='./Best_LA_model_for_DF.pth' --eval_output='eval_pre_trained_model_CM_scores_file_SSL_DF.txt' --database_path='./VCTK-synthesis-test-2_best' --batch_size=256 

Device: cuda
nb_params: 317837834
Model loaded : ./Best_LA_model_for_DF.pth
no. of eval trials 611829
/home/hungdx/miniconda3/envs/SSL_Spoofing/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
Traceback (most recent call last):
  File "/home/hungdx/miniconda3/envs/SSL_Spoofing/lib/python3.7/site-packages/librosa/core/audio.py", line 155, in load
    context = sf.SoundFile(path)
  File "/home/hungdx/miniconda3/envs/SSL_Spoofing/lib/python3.7/site-packages/soundfile.py", line 658, in __init__
    self._file = self._open(file, mode_int, closefd)
  File "/home/hungdx/miniconda3/envs/SSL_Spoofing/lib/python3.7/site-packages/soundfile.py", line 1216, in _open
    raise LibsndfileError(err, prefix="Error opening {0!r}: ".format(self.name))
soundfile.LibsndfileError: Error opening './VCTK-synthesis-test-2_bestASVspoof2021_DF_eval/flac/DF_E_2000011.flac': System error.

During handling of the above 